# Cyclic Polling System with Exhaustive Service and Exponential Times

This example demonstrates a cyclic polling system with exhaustive service discipline and exponential arrival and service times. The two classes have different service rates.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
val model = Network("M[2]/M[2]/1-Exhaustive")

// Block 1: nodes
val source = Source(model, "mySource")
val queue = Queue(model, "myQueue", SchedStrategy.POLLING)
val sink = Sink(model, "mySink")

In [ ]:
// Block 2: classes with exponential processes
val oclass1 = OpenClass(model, "myClass1")
source.setArrival(oclass1, Exp(0.1))  // λ₁ = 0.1
queue.setService(oclass1, Exp(1.0))   // μ₁ = 1.0, so E[S₁] = 1.0

val oclass2 = OpenClass(model, "myClass2")
source.setArrival(oclass2, Exp(0.1))  // λ₂ = 0.1
queue.setService(oclass2, Exp(1.5))   // μ₂ = 1.5, so E[S₂] = 1/1.5 ≈ 0.667

// Configure polling system
queue.setPollingType(PollingType.EXHAUSTIVE)

In [ ]:
// Block 3: topology
val P = model.initRoutingMatrix()
P.addRoute(oclass1, source, queue, sink)
P.addRoute(oclass2, source, queue, sink)
model.link(P)

In [ ]:
// Calculate theoretical utilizations
val lambda1 = 0.1
val lambda2 = 0.1
val mu1 = 1.0
val mu2 = 1.5

val rho1 = lambda1 / mu1  // ρ₁ = 0.1 / 1.0 = 0.1
val rho2 = lambda2 / mu2  // ρ₂ = 0.1 / 1.5 ≈ 0.067
val rhoTotal = rho1 + rho2

println("=== System Load Analysis ===")
println("Class 1: λ₁ = $lambda1, μ₁ = $mu1, ρ₁ = $rho1")
println("Class 2: λ₂ = $lambda2, μ₂ = $mu2, ρ₂ = $rho2")
println("Total utilization: ρ = $rhoTotal")
println("System is ${if (rhoTotal < 1.0) "stable" else "unstable"} (ρ ${if (rhoTotal < 1.0) "<" else "≥"} 1)")

In [ ]:
// Solve with MVA (solution is approximate in general for polling systems)
try {
    val mvaSolver = MVA(model)
    val avgTableMVA = mvaSolver.avgTable
    println("\nMVA Results:")
    avgTableMVA.print()
} catch (e: Exception) {
    println("\nMVA failed for this polling model: ${e.message}")
    println("This is expected as MVA may not support all polling configurations.")
}

In [ ]:
// Try solving with JMT
try {
    val jmtOptions = JMT.defaultOptions()
    jmtOptions.samples = 100000
    
    val jmtSolver = JMT(model, jmtOptions)
    val avgTableJMT = jmtSolver.avgTable
    println("\nJMT Results:")
    avgTableJMT.print()
} catch (e: Exception) {
    println("\nJMT failed for this polling model: ${e.message}")
    println("This is expected as JMT may not support all polling configurations.")
}

In [ ]:
// Display detailed model configuration
println("\n=== Detailed Polling System Configuration ===")
println("Model: ${model.name}")
println("Polling Type: Exhaustive")
println("Number of Classes: ${model.numberOfClasses}")
println("\nTraffic Characteristics:")
println("- Class 1: Poisson arrivals (λ=0.1), Exponential service (μ=1.0)")
println("- Class 2: Poisson arrivals (λ=0.1), Exponential service (μ=1.5)")
println("\nService Strategy:")
println("- Exhaustive: Server completely exhausts one class before switching")
println("- No explicit switchover times configured (defaults may apply)")
println("\nExpected Behavior:")
println("- Class 2 has faster service (higher μ) so shorter service times")
println("- Both classes have equal arrival rates")
println("- Exhaustive service can cause starvation if one class has high arrivals")
println("- System is stable with total ρ = $rhoTotal < 1")

This example demonstrates:

1. **Cyclic Polling with Exponential Timing**: 
   - Two job classes with Poisson arrivals (λ = 0.1 each)
   - Different exponential service rates (μ₁ = 1.0, μ₂ = 1.5)
   - Exhaustive service discipline
2. **Load Balancing Considerations**:
   - Class 1: ρ₁ = 0.1 (load = 0.1)
   - Class 2: ρ₂ ≈ 0.067 (load ≈ 0.067)
   - Total system load: ρ ≈ 0.167 (stable system)
3. **Exhaustive Service Dynamics**:
   - Server serves all jobs of one class before switching
   - Can lead to temporary starvation of the other class
   - Service time differences affect switching frequency
4. **Solver Challenges**:
   - Polling systems are analytically complex
   - Standard MVA and JMT may not support all polling features
   - Specialized polling analysis methods may be required

This model is relevant for systems like:
- Token ring networks
- Manufacturing systems with shared resources
- Database systems with different priority classes
- Communication protocols with round-robin scheduling